# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
# Загрузим все необходимые библиотеки:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings 
warnings.filterwarnings('ignore')

In [3]:
# Выведем данные:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

data = [data_1, data_1, data_3]
for i in data:
    i.info()
    display(i.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [4]:
# Удалим столбцы id, тк они будут мешать модели:
data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)
data_3 = data_3.drop('id', axis=1)

## Обучение и проверка модели

In [5]:
# Извлечем признаки и целевой признак:
features_1 = data_1.drop(['product'], axis=1)  
target_1 = data_1['product']

# Разделим данные на тренировочную и валидационную выборки (75/25): 
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

# Распечатаем размеры выборок:
df = features_train_1, target_train_1, features_valid_1, target_valid_1

for i in df:
    display(i.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [6]:
# Извлечем признаки и целевой признак:
features_2 = data_2.drop(['product'], axis=1)  
target_2 = data_2['product']

# Разделим данные на тренировочную и валидационную выборки (75/25):
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

# Распечатаем размеры выборок:
df = features_train_2, target_train_2, features_valid_2, target_valid_2

for i in df:
    display(i.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [7]:
# Извлечем признаки и целевой признак:
features_3 = data_3.drop(['product'], axis=1)  
target_3 = data_3['product']

# Разделим данные на тренировочную и валидационную выборки (75/25):
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size=0.25, random_state=12345)

# Распечатаем размеры выборок:
df = features_train_3, target_train_3, features_valid_3, target_valid_3

for i in df:
    display(i.shape)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

In [8]:
# Применим линейную регрессию для создания моделей:
model_1 = LinearRegression() 
model_1.fit(features_train_1, target_train_1)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
# Сделаем предсказания на валидационной выборке:
predicted_valid_1 = model_1.predict(features_valid_1)
predicted_valid_2 = model_2.predict(features_valid_2)
predicted_valid_3 = model_3.predict(features_valid_3)

In [10]:
# Рассчитаем средний предсказанный запас сырья и RMSE моделей:
def mean_rmse(predict, target):
    print('Mean =', predict.mean())
    mse = mean_squared_error(target, predict)
    print('RMSE =', np.sqrt(mse))

print('Region_1')
display(mean_rmse(predicted_valid_1, target_valid_1))
print('Region_2')
display(mean_rmse(predicted_valid_2, target_valid_2))
print('Region_3')
mean_rmse(predicted_valid_3, target_valid_3)

Region_1
Mean = 92.59256778438038
RMSE = 37.5794217150813


None

Region_2
Mean = 68.728546895446
RMSE = 0.893099286775616


None

Region_3
Mean = 94.96504596800489
RMSE = 40.02970873393434


**Вывод:** В регионе 2 самый низкий средний запас предсказанного сырья, при этом самые высокие значения RMSE в регионах 1 и 3, что говорит о высокой степени ошибки в данных регионах.

## Подготовка к расчёту прибыли

In [11]:
# При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
COUNT = 200
# Бюджет на разработку скважин в регионе — 10 млрд рублей.
BUDGET = 10000000000
# При нынешних ценах один баррель сырья приносит 450 рублей дохода. 
# Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
INCOME = 450000

In [12]:
# Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:
# Объем сырья * стоимость одной единицы продукции = бюджет разработки / количество скважин
volume = BUDGET / (COUNT * INCOME)
print(volume)

111.11111111111111


* Точка безубыточности по объему сырья для одной скважины составляет 111 тыс. бареллей.
* Регион 1 имеет среднее предсказанное значение 93, Регион 2 - 69, Регион 3 - 95.
* **Вывод: Среднее предсказанное значение во всех трех регионах ниже точки безубыточности для одной скважины. Необходимо отобрать лучшие скважины.**

## Расчёт прибыли и рисков 

In [13]:
# Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
def revenue(target, predictions):
    predict_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected_target = target[predict_sorted.index][:COUNT]
    return selected_target.sum() * INCOME - BUDGET

In [14]:
# Рассчитаем прибыль по каждому региону:
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)
target_valid_3 = target_valid_3.reset_index(drop=True)

print('Region_1')
display(revenue(target_valid_1, predicted_valid_1))
print('Region_2')
display(revenue(target_valid_2, predicted_valid_2))
print('Region_3')
revenue(target_valid_3, predicted_valid_3)

Region_1


3320826043.1398525

Region_2


2415086696.681511

Region_3


2710349963.5998325

In [15]:
# Применим технику Bootstrap с 1000 выборок, чтобы найти среднюю прибыль, 95%-й доверительный интервал и риск убытков:
def count_revenue(target, predict):
    state = np.random.RandomState(100)
    values = []
    for i in range(1000):
        target_sample = pd.Series(target).sample(n=500, random_state=state, replace=True)
        prob_sample = pd.Series(predict)[target_sample.index]
        rev_sample = revenue(target_sample, prob_sample)
        values.append(rev_sample)
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    #count = 0
    #for i in range(len(values)):
    #    if values[i] < 0:
    #        count += 1 count/len(values)*100
    return values.mean(), lower, upper, (values < 0).mean()*100

In [16]:
print('Region_1')
display(count_revenue(target_valid_1, predicted_valid_1))
print('Region_2')
display(count_revenue(target_valid_2, predicted_valid_2))
print('Region_3')
count_revenue(target_valid_3, predicted_valid_3)

Region_1


(408417421.9472264, -122256880.33374405, 989778917.6915294, 6.800000000000001)

Region_2


(504932744.04616815, 95193084.76780239, 916496786.6306496, 1.0)

Region_3


(418317134.1329915, -159796161.96567866, 947337201.5616112, 7.8)

**Вывод:** Наилучшим по показателям оказался Регион 2, в котором максимальная средняя прибыль среди трех регионов -505 млн. и риск отрицательной прибыли минимальный - 1%.